# 3-sphere: the round metric

This worksheet demonstrates a few capabilities of
[SageManifolds](http://sagemanifolds.obspm.fr) (version 1.0, as included in SageMath 7.5)
on the example of the 3-dimensional sphere, $\mathbb{S}^3$.

Click [here](https://raw.githubusercontent.com/sagemanifolds/SageManifolds/master/Worksheets/v1.0/SM_sphere_S3_metric.ipynb) to download the worksheet file (ipynb format). To run it, you must start SageMath with the Jupyter notebook, via the command `sage -n jupyter`

*NB:* a version of SageMath at least equal to 7.5 is required to run this worksheet:

In [ ]:
version()

First we set up the notebook to display mathematical objects using LaTeX formatting:

In [ ]:
%display latex

We also define a viewer for 3D plots (use `'threejs'` or `'jmol'` for interactive 3D graphics):

In [ ]:
viewer3D = 'threejs' # must be 'threejs', jmol', 'tachyon' or None (default)

## $\mathbb{S}^3$ as a 3-dimensional differentiable manifold

We start by declaring $\mathbb{S}^3$ as a differentiable manifold of dimension 3 over $\mathbb{R}$:

In [ ]:
S3 = Manifold(3, 'S^3', latex_name=r'\mathbb{S}^3', start_index=1)

The first argument, `3`, is the dimension of the manifold, while the second argument is the symbol used to label the manifold, with the LaTeX output specified by the argument `latex_name`. The argument `start_index` sets the index range to be used on the manifold for labelling components w.r.t. a basis or a frame: `start_index=1` corresponds to $\{1,2,3\}$; the default value is `start_index=0`, yielding to $\{0,1,2\}$.

In [ ]:
print(S3)

In [ ]:
S3

### Coordinate charts on $\mathbb{S}^3$

The 3-sphere cannot be covered by a single chart. At least two charts are necessary, for instance the charts associated with the stereographic projections from two distinct points, $N$ and $S$ say,
which we may call the *North pole* and the *South pole* respectively. Let us introduce the open subsets covered by these two charts: 
$$ U := \mathbb{S}^3\setminus\{N\} $$  
$$ V := \mathbb{S}^3\setminus\{S\} $$

In [ ]:
U = S3.open_subset('U') ; print(U)

In [ ]:
V = S3.open_subset('V') ; print(V)

We declare that $\mathbb{S}^3 = U \cup V$:

In [ ]:
S3.declare_union(U, V)

Then we introduce the stereographic chart on $U$, denoting by $(x,y,z)$ the coordinates resulting from the stereographic projection from the North pole onto the equatorial plane:

In [ ]:
stereoN.<x,y,z> = U.chart()
stereoN

In [ ]:
stereoN.coord_range()

Similarly, we introduce on $V$ the coordinates $(x',y',z')$ corresponding to the stereographic projection from the South pole onto the equatorial plane:

In [ ]:
stereoS.<xp,yp,zp> = V.chart("xp:x' yp:y' zp:z'")
stereoS

In [ ]:
stereoS.coord_range()

We have to specify the **transition map** between the charts `stereoN` = $(U,(x,y,z))$ and `stereoS` = $(V,(x',y',z'))$; it is given by the standard inversion formulas:

In [ ]:
r2 = x^2+y^2+z^2
stereoN_to_S = stereoN.transition_map(stereoS, 
                                      (x/r2, y/r2, z/r2), 
                                      intersection_name='W',
                                      restrictions1= x^2+y^2+z^2!=0, 
                                      restrictions2= xp^2+yp^2+zp^2!=0)
stereoN_to_S.display()

In the above declaration, `'W'` is the name given to the open subset where the two charts overlap: $W := U\cap V$, the condition $x^2+y^2+z^2\not=0$  defines $W$ as a subset of $U$, and the condition $x'^2+y'^2+z'^2\not=0$ defines $W$ as a subset of $V$.

The inverse coordinate transformation is computed by means of the method `inverse()`:

In [ ]:
stereoS_to_N = stereoN_to_S.inverse()
stereoS_to_N.display()

Note that the situation is of course perfectly symmetric regarding the coordinates $(x,y,z)$ and $(x',y',z')$.

At this stage, the user's atlas has four charts:

In [ ]:
S3.atlas()

For future reference, we store $W=U\cap V$ into a Python variable:

In [ ]:
W = U.intersection(V)
print(W)

### The North and South poles

$N$ is the point of $V$ of stereographic coordinates $(x',y',z')=(0,0,0)$:

In [ ]:
N = V((0,0,0), chart=stereoS, name='N')
print(N)

while $S$ is the point of U of stereographic coordinates $(x,y,z)=(0,0,0)$:

In [ ]:
S = U((0,0,0), chart=stereoN, name='S')
print(S)

We have of course

In [ ]:
all([N not in U, N in V, S in U, S not in V])

## Embedding of $\mathbb{S}^3$ into $\mathbb{R}^4$

Let us first declare $\mathbb{R}^4$ as a 4-dimensional manifold covered by a single chart (the so-called **Cartesian coordinates**):

In [ ]:
R4 = Manifold(4, 'R^4', r'\mathbb{R}^4')
X4.<T,X,Y,Z> = R4.chart()
X4

The embedding of $\mathbb{S}^3$ into $\mathbb{R}^4$ is then defined by the standard formulas relating the stereographic coordinates to the ambient Cartesian ones when considering a **stereographic projection** from the point $(-1,0,0,0)$ to the equatorial plane $T=0$:

In [ ]:
rp2 = xp^2 + yp^2 + zp^2
Phi = S3.diff_map(R4, {(stereoN, X4): 
                       [(1-r2)/(r2+1), 2*x/(r2+1), 
                        2*y/(r2+1), 2*z/(r2+1)],
                       (stereoS, X4):
                       [(rp2-1)/(rp2+1), 2*xp/(rp2+1), 
                        2*yp/(rp2+1), 2*zp/(rp2+1)]},
                  name='Phi', latex_name=r'\Phi')
Phi.display()

## Hyperspherical coordinates

The hyperspherical coordinates $(\chi, \theta, \phi)$ generalize the standard spherical coordinates $(\theta, \phi)$ on $\mathbb{S}^2$. They are defined on the open domain $A\subset W \subset \mathbb{S}^3$ that is the complement of the "origin meridian"; since the latter is defined by $y=0$ and $x\geq 0$, we declare:

In [ ]:
A = W.open_subset('A', coord_def={stereoN.restrict(W): (y!=0, x<0), 
                                  stereoS.restrict(W): (yp!=0, xp<0)})
print(A)

In [ ]:
spher.<ch,th,ph> = A.chart(r'ch:(0,pi):\chi th:(0,pi):\theta ph:(0,2*pi):\phi')
spher

In [ ]:
den = 1 + cos(ch)
spher_to_stereoN = spher.transition_map(stereoN.restrict(A), 
                                        (sin(ch)*sin(th)*cos(ph)/den,
                                         sin(ch)*sin(th)*sin(ph)/den,
                                         sin(ch)*cos(th)/den))
spher_to_stereoN.display()

In [ ]:
spher_to_stereoN.set_inverse(2*atan(sqrt(x^2+y^2+z^2)),
                             atan2(sqrt(x^2+y^2), z),
                             atan2(-y, -x)+pi,
                             verbose=True)

In [ ]:
spher_to_stereoN.inverse().display()

In [ ]:
spher_to_stereoS = stereoN_to_S.restrict(A) * spher_to_stereoN
spher_to_stereoS.display()

In [ ]:
stereoS_to_spher = spher_to_stereoN.inverse() * stereoS_to_N.restrict(A)
stereoS_to_spher.display()

In [ ]:
Phi.display(stereoN.restrict(A), X4)

In [ ]:
Phi.display(spher, X4)

In [ ]:
Phi.display()

## Global left-invariant vector frame on $\mathbb{S}^3$

The vector frames associated with the two stereographic charts are

In [ ]:
frameN = stereoN.frame()
frameN

In [ ]:
frameS = stereoS.frame()
frameS

None of these two frames cover entirely the 3-sphere, since $U$ and $V$ are strict subsets of $\mathbb{S}^3$. Now, as it is well known, $\mathbb{S}^3$ admits global vector frames, i.e. $\mathbb{S}^3$ is a **parallelizable manifold**. Among all the spheres, it shares this remarkable property with $\mathbb{S}^1$ and $\mathbb{S}^7$. We shall use a global vector frame $(\mathbb{S}^3, (\epsilon_1, \epsilon_2, \epsilon_3))$ associated with the Lie group structure of $\mathbb{S}^3$, namely the left-invariant vector frame constructed in the 
[worksheet devoted to vector fields on $\mathbb{S}^3$](http://nbviewer.jupyter.org/github/sagemanifolds/SageManifolds/blob/master/Worksheets/v1.0/SM_sphere_S3_vectors.ipynb). We first declare this vector frame on all $\mathbb{S}^3$:

In [ ]:
E = S3.vector_frame('E', latex_symbol=r'\epsilon')
E

On $U$, we relate this frame to the stereographic coordinate frame, by means of the formulas obtained in the vector field worksheet mentionned above:

In [ ]:
E_U = E.restrict(U); E_U

In [ ]:
E_U[1][frameN,:,stereoN] = \
    [(x^2-y^2-z^2+1)/2, x*y+z, x*z-y]

In [ ]:
E_U[2][frameN,:,stereoN] = \
    [x*y-z, (1-x^2+y^2-z^2)/2, x+y*z]

In [ ]:
E_U[3][frameN,:,stereoN] = \
    [x*z+y, y*z-x, (1-x^2-y^2+z^2)/2]

Similarly, on $V$, we relate the global frame with the stereographic coordinate frame from the South pole:

In [ ]:
E_V = E.restrict(V); E_V

In [ ]:
E_V[1][frameS,:, stereoS] = \
    [(yp^2+zp^2-xp^2-1)/2, zp-xp*yp, -yp-xp*zp]

In [ ]:
E_V[2][frameS,:, stereoS] = \
    [-zp-xp*yp, (xp^2-yp^2+zp^2-1)/2, xp-yp*zp]

In [ ]:
E_V[3][frameS,:, stereoS] = \
    [yp-xp*zp, -xp-yp*zp, (xp^2+yp^2-zp^2-1)/2]

Let us display the links between the various frames:

In [ ]:
for i in S3.irange():
    show(E[i].display(frameN))
print(" ")
for i in S3.irange():
    show(E[i].display(frameS))

To complete the links, we introduce the change-of-frame operators: first the operator $P$ such that, on $U$, $\epsilon_i = P(\partial/\partial{x^i})$, with $x^i=(x,y,z)$:

In [ ]:
P = U.automorphism_field()
for i in S3.irange():
    for j in S3.irange():
        P[j,i] = E_U[i][j]

In [ ]:
all([E_U[i] == P(frameN[i]) for i in S3.irange()])

We add $P$ to the known changes of frame on $U$:

In [ ]:
U.set_change_of_frame(frameN, E_U, P)

Similarly, on $V$, we introduce the operator $P$ such that $\epsilon_i = P(\partial/\partial {x'}^i)$, with ${x'}^i=(x',y',z')$:

In [ ]:
P = V.automorphism_field()
for i in S3.irange():
    for j in S3.irange():
        P[j,i] = E_V[i][j]

In [ ]:
all([E_V[i] == P(frameS[i]) for i in S3.irange()])

In [ ]:
V.set_change_of_frame(frameS, E_V, P)

## Hopf coordinates

Despite there does not exist any coordinate chart associated to the global vector frame $(\mathbb{S}^3, (\epsilon_1, \epsilon_2, \epsilon_3))$, there exist a chart, called the
**Hopf chart**, which has some link to it in the sense that some coordinate lines are integral curves of $\epsilon_3$. The Hopf coordinates have been introduced in the [worksheet devoted to the Hopf fibration](http://nbviewer.jupyter.org/github/sagemanifolds/SageManifolds/blob/master/Worksheets/v1.0/SM_sphere_S3_Hopf.ipynb): 

In [ ]:
B = U.open_subset('B', coord_def={stereoN.restrict(U): 
                                   [x^2+y^2!=0, x^2+y^2+z^2!=1, 
                                    (1-x^2-y^2-z^2)*x-2*y*z!=0]})
print(B)

In [ ]:
Hcoord.<eta,alp,bet> = B.chart(r"eta:(0,pi/2):\eta alpha:(0,2*pi):\alpha beta:(0,2*pi):\beta")
Hcoord

In [ ]:
Hcoord_to_stereoN = Hcoord.transition_map(
                        stereoN.restrict(U),
                        (sin(eta)*cos(alp+bet)/(1+cos(eta)*sin(alp)),
                         sin(eta)*sin(alp+bet)/(1+cos(eta)*sin(alp)),
                         cos(eta)*cos(alp)/(1+cos(eta)*sin(alp))))
Hcoord_to_stereoN.display()

In [ ]:
Hcoord_to_stereoN.set_inverse(asin(2*sqrt(x^2+y^2)/(1+x^2+y^2+z^2)),
                              atan2(x^2+y^2+z^2-1, -2*z) + pi,
                              atan2(-y,-x) - atan2(x^2+y^2+z^2-1, -2*z),
                              verbose=True)

In [ ]:
Hcoord_to_stereoN.inverse().display()

In [ ]:
E_U[1].display(Hcoord.frame(), Hcoord)

In [ ]:
E_U[2].display(Hcoord.frame(), Hcoord)

In [ ]:
E_U[3].display(Hcoord.frame(), Hcoord)

In [ ]:
One = S3((0,0,0), chart=stereoN)

In [ ]:
E[1].at(One).display()

In [ ]:
E[2].at(One).display()

In [ ]:
E_U[3].at(One).display()

## Round metric on $\mathbb{S}^3$

The standard metric on $\mathbb{S}^3$ is that induced by the Euclidean metric of $\mathbb{R}^4$. Let us start by defining the latter:

In [ ]:
h = R4.metric('h')
h[0,0], h[1,1], h[2,2], h[3, 3] = 1, 1, 1, 1
h.display()

The metric $g$ on $\mathbb{S}^3$ is the pullback of $h$ by the embedding $\Phi$:

In [ ]:
g = S3.metric('g')
g.set( Phi.pullback(h) )
print(g)

In [ ]:
g.display(frameN)

In [ ]:
for i in S3.irange():
    g[frameN, i,i].factor()
g.display(frameN)

In [ ]:
g.display(frameS)

In [ ]:
for i in S3.irange():
    g[frameS, i,i].factor()
g.display(frameS)

In [ ]:
g.display(E_U)

In [ ]:
g.display(E_V)

In [ ]:
for i in S3.irange():
    g.add_comp(E)[i, i] = 1
g.display(E)

In [ ]:
g.display(spher.frame(), spher)

In [ ]:
g.display(Hcoord.frame(), Hcoord)

### The Christoffel symbols

In [ ]:
g.christoffel_symbols_display(stereoN)

### The Riemann tensor

In [ ]:
Riem = g.riemann()
print(Riem)

In [ ]:
Riem.display_comp(frameN, stereoN, only_nonredundant=True)

In [ ]:
Riem.display_comp(spher.frame(), spher, only_nonredundant=True)

In [ ]:
Riem.display_comp(Hcoord.frame(), Hcoord, only_nonredundant=True)

In [ ]:
Riem.display_comp(E_U, stereoN, only_nonredundant=True)

In [ ]:
Riem.display(E_U)

### The Ricci tensor

In [ ]:
Ric = g.ricci()
print(Ric)

In [ ]:
Ric.display(frameN)

In [ ]:
for i in S3.irange():
    Ric[frameN, i,i].factor()
Ric.display(frameN)

In [ ]:
Ric.restrict(U) == 2 * g.restrict(U)

### The Ricci scalar

In [ ]:
R_A = g.restrict(A).ricci_scalar()
print(R_A)

In [ ]:
R_A.display()

## The volume 3-form (Levi-Civita tensor)

In [ ]:
eps = g.volume_form()
print(eps)
eps.display()

In [ ]:
eps.display(frameN, stereoN)

In [ ]:
print(eps.parent())

In [ ]:
eps._components

In [ ]:
U._covering_frames

In [ ]:
W._covering_frames

In [ ]:
epsU = eps.restrict(U)
epsU.display()

In [ ]:
epsU.display(spher.frame(), spher)

In [ ]:
s = epsU[spher.frame(), 1, 2, 3, spher].expr()
s

In [ ]:
s1 = s.trig_reduce().simplify_full().simplify_full()
s1

In [ ]:
s2 = (s1 / (sin(ch)^2*sin(th))).simplify_full()
s2

In [ ]:
s2.trig_expand()

In [ ]:
epsU.display(Hcoord.frame(), Hcoord)